In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Fungsi Utilitas untuk Aljabar Linear "Hardcore"
def svd_analysis(matrix, k_components, title):
    print(f"\n--- ANALISIS ALJABAR LINEAR: {title} ---")
    
    # 1. Representasi Matriks (m x n)
    m, n = matrix.shape
    print(f"Dimensi Matriks Awal (A): {m} x {n}")
    
    # 2. Centering / Mean Subtraction (Penting untuk PCA/SVD)
    # Rumus: X_centered = X - mean(X)
    mean_vec = np.mean(matrix, axis=0)
    matrix_centered = matrix - mean_vec
    
    # 3. Dekomposisi SVD Murni
    # Rumus: A = U . Sigma . Vt
    U, S, Vt = np.linalg.svd(matrix_centered, full_matrices=False)
    
    # Tampilkan Struktur Matriks Hasil Dekomposisi
    print(f"Matriks U (Orthogonal Basis): {U.shape}")
    print(f"Vektor Singular Values (Sigma): {S.shape}")
    print(f"Matriks V^T (Feature Weights): {Vt.shape}")
    
    # 4. Hitung Explained Variance Ratio (EVR) secara Manual
    # Rumus: Variansi_i = (S_i^2) / (m-1)
    eigenvalues = (S ** 2) / (m - 1)
    total_variance = np.sum(eigenvalues)
    explained_variance_ratio = eigenvalues / total_variance
    cumulative_variance = np.cumsum(explained_variance_ratio)
    
    print(f"Variansi Terjelaskan oleh {k_components} Komponen: {cumulative_variance[k_components-1]*100:.2f}%")
    
    # 5. Rekonstruksi Low-Rank (Approximation)
    # Rumus: A_k = U_k . Sigma_k . Vt_k
    U_k = U[:, :k_components]
    S_k = np.diag(S[:k_components]) # Ubah vektor ke matriks diagonal
    Vt_k = Vt[:k_components, :]
    
    matrix_reconstructed = np.dot(np.dot(U_k, S_k), Vt_k) + mean_vec
    
    # 6. Hitung Error Rekonstruksi (Frobenius Norm)
    # Rumus: ||A - A_k||_F
    error = np.linalg.norm(matrix - matrix_reconstructed, 'fro')
    print(f"Reconstruction Error (Frobenius Norm): {error:.4f}")
    
    return matrix_reconstructed

# ==========================================
# EKSEKUSI 4 STUDI KASUS
# ==========================================

# --- KASUS 1: EKONOMI (Reduksi Dimensi) ---
data_eco = pd.read_csv("data_ekonomi.csv").values
# Target: 90% variansi, biasanya tercapai di k kecil karena korelasi tinggi
_ = svd_analysis(data_eco, k_components=5, title="ST. KASUS 1: DATA EKONOMI")

# --- KASUS 2: SENSOR (Noise Removal) ---
data_sensor = pd.read_csv("data_sensor.csv").values
# Kita ambil k=3 karena sinyal asli sangat sederhana (sinus), sisanya noise
clean_sensor = svd_analysis(data_sensor, k_components=3, title="ST. KASUS 2: SENSOR NOISY")
# Bukti Noise Hilang: Bandingkan Standard Deviasi
print(f"Std Dev Awal (Noisy): {np.std(data_sensor):.4f}")
print(f"Std Dev Hasil Filter (Clean): {np.std(clean_sensor):.4f} (Lebih stabil)")

# --- KASUS 3: SAHAM (Feature Extraction) ---
data_stock = pd.read_csv("data_saham.csv").values
# Proyeksi ke ruang fitur baru (Z = X . V)
# Ini langkah preprocessing untuk input ke Neural Network
stock_reconstructed = svd_analysis(data_stock, k_components=8, title="ST. KASUS 3: FITUR SAHAM")

# --- KASUS 4: CITRA DIGITAL (Kompresi Matriks) ---
data_image = pd.read_csv("citra_digital.csv", header=None).values
# Kompresi Ekstrem: dari 512 rank menjadi 50 rank
img_compressed = svd_analysis(data_image, k_components=50, title="ST. KASUS 4: CITRA DIGITAL")

# Hitung Rasio Kompresi Memori
original_size = 512 * 512
compressed_size = (512*50) + 50 + (50*512) # U_k + S_k + Vt_k
print(f"Rasio Kompresi Data: {original_size / compressed_size:.2f}x Lebih Kecil")


--- ANALISIS ALJABAR LINEAR: ST. KASUS 1: DATA EKONOMI ---
Dimensi Matriks Awal (A): 200 x 50
Matriks U (Orthogonal Basis): (200, 50)
Vektor Singular Values (Sigma): (50,)
Matriks V^T (Feature Weights): (50, 50)
Variansi Terjelaskan oleh 5 Komponen: 95.03%
Reconstruction Error (Frobenius Norm): 9.3855

--- ANALISIS ALJABAR LINEAR: ST. KASUS 2: SENSOR NOISY ---
Dimensi Matriks Awal (A): 100 x 30
Matriks U (Orthogonal Basis): (100, 30)
Vektor Singular Values (Sigma): (30,)
Matriks V^T (Feature Weights): (30, 30)
Variansi Terjelaskan oleh 3 Komponen: 52.55%
Reconstruction Error (Frobenius Norm): 24.6459
Std Dev Awal (Noisy): 0.6575
Std Dev Hasil Filter (Clean): 0.4794 (Lebih stabil)

--- ANALISIS ALJABAR LINEAR: ST. KASUS 3: FITUR SAHAM ---
Dimensi Matriks Awal (A): 150 x 40
Matriks U (Orthogonal Basis): (150, 40)
Vektor Singular Values (Sigma): (40,)
Matriks V^T (Feature Weights): (40, 40)
Variansi Terjelaskan oleh 8 Komponen: 94.60%
Reconstruction Error (Frobenius Norm): 62.1462

--- A

In [5]:
import numpy as np
import pandas as pd

# 1. Memuat Dataset Ekonomi (200 wilayah x 50 indikator)
# Dataset disimulasikan memiliki korelasi internal yang tinggi
data_eco = pd.read_csv("data_ekonomi.csv").values

# 2. Standardisasi Data (Penting untuk SVD/PCA)
# Menghindari bias akibat perbedaan skala (misal: PDB vs Inflasi)
mean_vec = np.mean(data_eco, axis=0)
std_vec = np.std(data_eco, axis=0)
data_std = (data_eco - mean_vec) / std_vec

# 3. Eksekusi SVD (Singular Value Decomposition)
# A = U . S . Vt
U, S, Vt = np.linalg.svd(data_std, full_matrices=False)

# 4. Proyeksi ke Ruang Dimensi Rendah (k=5)
k = 5
# Matriks W adalah 5 baris pertama dari Vt (basis fitur baru)
W = Vt[:k, :].T 
data_reduced = np.dot(data_std, W)

print(f"Dimensi Awal: {data_std.shape}")
print(f"Dimensi Reduksi: {data_reduced.shape}")

# 5. Evaluasi Variansi (Cumulative Explained Variance)
# Variansi proporsional terhadap kuadrat nilai singular
eigenvalues = (S ** 2) / (data_std.shape[0] - 1)
total_variance = np.sum(eigenvalues)
explained_variance = np.sum(eigenvalues[:k]) / total_variance

print(f"Variansi Terjelaskan (k=5): {explained_variance*100:.2f}%")

Dimensi Awal: (200, 50)
Dimensi Reduksi: (200, 5)
Variansi Terjelaskan (k=5): 94.15%


In [6]:
import numpy as np
import pandas as pd

# 1. Memuat Dataset Sensor Noisy
# Data ini mengandung pola sinyal sinus yang tertutup noise Gaussian
data_sensor = pd.read_csv("data_sensor.csv").values

# 2. Dekomposisi SVD
# Algoritma memisahkan struktur data menjadi komponen ortogonal
U, S, Vt = np.linalg.svd(data_sensor, full_matrices=False)

# 3. Rekonstruksi Low-Rank (Filtering)
# Kita ambil k=3 karena sinyal fisik biasanya berdimensi rendah
k = 3
S_clean = np.diag(S[:k])
U_clean = U[:, :k]
Vt_clean = Vt[:k, :]

# A_clean = U_k . S_k . Vt_k
data_filtered = np.dot(np.dot(U_clean, S_clean), Vt_clean)

# 4. Evaluasi Kuantitatif (Standard Deviation Reduction)
std_original = np.std(data_sensor)
std_filtered = np.std(data_filtered)
noise_reduction = (1 - (std_filtered / std_original)) * 100

print(f"Deviasi Standar Awal (Sinyal + Noise): {std_original:.4f}")
print(f"Deviasi Standar Akhir (Sinyal Murni): {std_filtered:.4f}")
print(f"Reduksi Variansi Noise: {noise_reduction:.2f}%")

Deviasi Standar Awal (Sinyal + Noise): 0.6575
Deviasi Standar Akhir (Sinyal Murni): 0.4769
Reduksi Variansi Noise: 27.46%


In [7]:
import numpy as np
import pandas as pd

# 1. Memuat Dataset Saham (150 hari x 40 indikator)
# Data ini memiliki multikolinearitas tinggi (banyak indikator mirip)
data_stock = pd.read_csv("data_saham.csv").values

# 2. Standardisasi (Z-Score)
# Penting agar indikator dengan satuan berbeda (Rp vs %) setara
mean_vec = np.mean(data_stock, axis=0)
std_vec = np.std(data_stock, axis=0)
data_std = (data_stock - mean_vec) / std_vec

# 3. Dekomposisi SVD
U, S, Vt = np.linalg.svd(data_std, full_matrices=False)

# 4. Konstruksi Matriks Proyeksi (k=8)
# Kita mereduksi fitur sebesar 80% (dari 40 ke 8)
k = 8
W = Vt[:k, :].T  # Transpose dari Vt untuk mendapatkan bobot proyeksi

# 5. Transformasi ke Ruang Fitur Baru (Latent Space)
# Z = X . W
features_extracted = np.dot(data_std, W)

print(f"Dimensi Awal (Fitur Teknikal): {data_std.shape}")
print(f"Dimensi Akhir (Latent Features): {features_extracted.shape}")

# 6. Hitung Retensi Informasi
eigenvalues = (S ** 2) / (data_stock.shape[0] - 1)
explained_variance = np.sum(eigenvalues[:k]) / np.sum(eigenvalues)
print(f"Informasi Pasar yang Dipertahankan: {explained_variance*100:.2f}%")

Dimensi Awal (Fitur Teknikal): (150, 40)
Dimensi Akhir (Latent Features): (150, 8)
Informasi Pasar yang Dipertahankan: 94.53%
